In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import glob
from skimage.io import imread_collection
import os
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
import imageio as iio
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
keras = tf.keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.applications import DenseNet121
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.merge import concatenate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix, confusion_matrix  

#Load Dataset



In [ ]:
#Create labels for the images. 
tr_labels = []
i = 677
while(i>0):
  tr_labels.append(np.array([1])) #1 for actual face. 
  i = i-1
i = 446
while(i>0):
  tr_labels.append(np.array([0])) #0 for food. 
  i = i-1

ts_labels = []
i = 40
while(i>0):
  ts_labels.append(np.array([0])) #0 for food that appears as faces  
  i = i-1

In [ ]:
print(len(tr_labels), len(ts_labels))

1123 40


In [ ]:
tr_labels = np.array(tr_labels)
tr_labels.shape
ts_labels = np.array(ts_labels)

In [ ]:
#Read all the images from dataset. 
tr_images = []
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Train/Face/*.jpeg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Train/Face/*.jpg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Train/Face/*.png"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Train/Food/*.jpeg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Train/Food/*.jpg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Train/Food/*.png"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)


ts_images = []
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Test/*.jpeg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    ts_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Test/*.jpg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    ts_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Food/Test/*.png"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    ts_images.append(n)


In [ ]:
tr_images = np.array(tr_images)
ts_images = np.array(ts_images)
print(tr_images.shape)
print(ts_images.shape)
print(len(tr_labels))
print(len(ts_labels))
print(type(tr_labels))

(1123, 224, 224, 3)
(40, 224, 224, 3)
1123
40
<class 'numpy.ndarray'>


In [ ]:
#Data Augmentation

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=30,  # randomly rotate images upto 180 degrees
        shear_range=0.2,
        zoom_range = 0.15, # randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=False, # randomly flip images vertically
        fill_mode='nearest')  


datagen.fit(tr_images)

#CNN Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 173056)           

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

hist = model.fit(tr_images, tr_labels, epochs=10, 
                    validation_data=(ts_images, ts_labels))

Epoch 1/10
36/36 [==============================] - 14s 61ms/step - loss: 251.5946 - accuracy: 0.6287 - val_loss: 2.8160 - val_accuracy: 0.2500
Epoch 2/10
36/36 [==============================] - 2s 49ms/step - loss: 0.2955 - accuracy: 0.8994 - val_loss: 0.5814 - val_accuracy: 0.7500
Epoch 3/10
36/36 [==============================] - 2s 50ms/step - loss: 0.6883 - accuracy: 0.9110 - val_loss: 2.6825 - val_accuracy: 0.5750
Epoch 4/10
36/36 [==============================] - 2s 49ms/step - loss: 0.2697 - accuracy: 0.9154 - val_loss: 1.1341 - val_accuracy: 0.6500
Epoch 5/10
36/36 [==============================] - 2s 50ms/step - loss: 0.2572 - accuracy: 0.9261 - val_loss: 0.9953 - val_accuracy: 0.8500
Epoch 6/10
36/36 [==============================] - 2s 48ms/step - loss: 0.1298 - accuracy: 0.9617 - val_loss: 0.8224 - val_accuracy: 0.8000
Epoch 7/10
36/36 [==============================] - 2s 49ms/step - loss: 0.0921 - accuracy: 0.9760 - val_loss: 1.2931 - val_accuracy: 0.8750
Epoch 8/10

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 1.5009 - accuracy: 0.5250 - 42ms/epoch - 21ms/step
0.5249999761581421


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[-1.0207621e+00  3.3273014e-01]
 [-8.5569108e-01  7.6485258e-01]
 [ 1.0494262e+00 -1.8533471e+00]
 [ 3.3600271e-01 -1.6393757e-01]
 [-5.9717499e-02 -6.6198960e-02]
 [ 1.5341399e+00 -2.0643208e+00]
 [ 2.3690047e+00 -2.6446183e+00]
 [-1.9948529e+00 -7.4521670e+00]
 [ 4.3861054e-02 -4.8700057e-02]
 [-2.8665404e+00 -4.3714318e+00]
 [-2.6823058e+00  2.3335781e+00]
 [-1.7505158e-02  3.4993723e-02]
 [ 9.3761259e-01 -5.6124389e-01]
 [ 7.6897430e-01 -3.3460817e-01]
 [-2.2260347e-01 -2.1725544e-01]
 [ 1.4154264e+00 -9.1628808e-01]
 [-1.6508985e+00  5.7228230e-02]
 [-1.4504079e+00 -1.1086867e+00]
 [-7.4426281e-01  8.2432836e-01]
 [ 2.2610002e+00 -4.8919897e+00]
 [-2.3818510e+01  8.7793961e+00]
 [-1.6347907e-02 -1.7534062e-01]
 [-4.3703657e-02  5.0901767e-02]
 [-4.4344366e-02  4.7541931e-02]
 [-4.0962882e-02  5.3028028e-02]
 [-1.0252789e+00 -6.7227179e-01]
 [-1.1080418e-02  9.8565388e-03]
 [-4.0671381e-01 -1.1077168e+00]
 [-4.8288617e-02  4.6673540e-02]
 [-6.2337494e-01 -1.8024377e+00]
 [-4.46316

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Pre-Trained MobileVNet

In [ ]:
mobilenetV2 = MobileNetV2(input_shape=(224, 224, 3),
                          include_top=False,
                          weights='imagenet'
                          )

average_layer = GlobalAveragePooling2D()

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
model = Sequential([
    mobilenetV2,
    average_layer,
    Dense(256, activation=tf.nn.relu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation=tf.nn.relu),
    BatchNormalization(),
    Dropout(0.1),
    Dense(2, activation=tf.nn.softmax)
])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001 
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=5, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/5
36/36 [==============================] - 7s 81ms/step - loss: 0.3656 - accuracy: 0.8486 - val_loss: 1.1135 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/5
36/36 [==============================] - 2s 43ms/step - loss: 0.1296 - accuracy: 0.9564 - val_loss: 1.1045 - val_accuracy: 0.5250 - lr: 0.0010
Epoch 3/5
36/36 [==============================] - 2s 43ms/step - loss: 0.1252 - accuracy: 0.9501 - val_loss: 0.9805 - val_accuracy: 0.5750 - lr: 0.0010
Epoch 4/5
36/36 [==============================] - 2s 43ms/step - loss: 0.0799 - accuracy: 0.9751 - val_loss: 1.0622 - val_accuracy: 0.5750 - lr: 1.0000e-04
Epoch 5/5
36/36 [==============================] - 2s 43ms/step - loss: 0.0807 - accuracy: 0.9742 - val_loss: 0.9288 - val_accuracy: 0.6250 - lr: 1.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.9288 - accuracy: 0.6250 - 63ms/epoch - 32ms/step
0.625


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[7.2069645e-01 2.7930361e-01]
 [9.6569610e-01 3.4303911e-02]
 [9.2388397e-01 7.6116033e-02]
 [5.6942869e-02 9.4305712e-01]
 [7.2018784e-01 2.7981216e-01]
 [9.6279681e-01 3.7203133e-02]
 [6.5845579e-01 3.4154427e-01]
 [7.4904673e-02 9.2509532e-01]
 [6.3472903e-01 3.6527097e-01]
 [5.9067738e-01 4.0932262e-01]
 [8.9385867e-01 1.0614131e-01]
 [8.1555939e-01 1.8444064e-01]
 [1.9287691e-02 9.8071229e-01]
 [7.3250848e-01 2.6749149e-01]
 [1.0645776e-01 8.9354217e-01]
 [9.5777512e-01 4.2224895e-02]
 [1.1568388e-01 8.8431609e-01]
 [9.9073458e-01 9.2654442e-03]
 [2.3098604e-01 7.6901394e-01]
 [3.7071399e-02 9.6292853e-01]
 [4.3789770e-02 9.5621026e-01]
 [9.9929893e-01 7.0101139e-04]
 [9.1077977e-01 8.9220211e-02]
 [9.7959560e-01 2.0404421e-02]
 [9.8046035e-01 1.9539608e-02]
 [9.8323959e-01 1.6760373e-02]
 [9.7300094e-01 2.6999040e-02]
 [8.4316623e-01 1.5683375e-01]
 [9.8439342e-01 1.5606536e-02]
 [8.2531732e-01 1.7468265e-01]
 [1.5808010e-02 9.8419201e-01]
 [1.4311543e-01 8.5688460e-01]
 [1.9979

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Pre-Trained VGG16

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg16_model.output[-1]

<KerasTensor: shape=(7, 7, 512) dtype=float32 (created by layer 'tf.__operators__.getitem')>

In [ ]:
model = Sequential([vgg16_model,
                    Flatten(),
#                     GlobalAveragePooling2D(),
#                     Dense(512, activation = "relu"),
#                     BatchNormalization(),
#                     Dropout(0.3),
#                     Dense(128, activation = "relu"),
#                     Dropout(0.1),
#                     # Dense(32, activation = "relu"),
#                     # Dropout(0.3),
                    Dense(2, activation = "softmax")])

In [ ]:
model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_7 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=5, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/5
36/36 [==============================] - 12s 236ms/step - loss: 0.2333 - accuracy: 0.9840 - val_loss: 1.2789 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 2/5
36/36 [==============================] - 6s 179ms/step - loss: 0.0884 - accuracy: 0.9982 - val_loss: 4.8980 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 3/5
36/36 [==============================] - 7s 182ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.7451 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 4/5
36/36 [==============================] - 6s 177ms/step - loss: 4.2461e-10 - accuracy: 1.0000 - val_loss: 0.7422 - val_accuracy: 0.9750 - lr: 1.0000e-04
Epoch 5/5
36/36 [==============================] - 6s 172ms/step - loss: 2.1231e-10 - accuracy: 1.0000 - val_loss: 0.7422 - val_accuracy: 0.9750 - lr: 1.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.7422 - accuracy: 0.9750 - 264ms/epoch - 132ms/step
0.9750000238418579


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 4.6484642e-23]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 9.6728083e-13]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.2774166e-13 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Pre-Trained VGG19

In [ ]:
vgg19_model = tf.keras.applications.vgg19.VGG19(include_top=True, weights="imagenet", input_shape=(224,224,3))

574726144/574710816 [==============================] - 3s 0us/step


In [ ]:
vgg19_model.output[-1]

<KerasTensor: shape=(1000,) dtype=float32 (created by layer 'tf.__operators__.getitem_1')>

In [ ]:
model = Sequential()
model.add(vgg19_model)
model.add(Flatten())

model.add(Dense(512, activation = "relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(128, activation = "relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.1))


model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 flatten_3 (Flatten)         (None, 1000)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               512512    
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                      

In [ ]:
model.layers[0].trainable = False
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=10, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/10
36/36 [==============================] - 9s 210ms/step - loss: 0.1415 - accuracy: 0.9492 - val_loss: 0.8719 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
36/36 [==============================] - 7s 204ms/step - loss: 0.0207 - accuracy: 0.9947 - val_loss: 0.8507 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/10
36/36 [==============================] - 7s 207ms/step - loss: 0.0193 - accuracy: 0.9947 - val_loss: 0.7257 - val_accuracy: 0.3000 - lr: 0.0010
Epoch 4/10
36/36 [==============================] - 8s 210ms/step - loss: 0.0089 - accuracy: 0.9964 - val_loss: 0.6358 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 5/10
36/36 [==============================] - 8s 216ms/step - loss: 0.0093 - accuracy: 0.9964 - val_loss: 0.5567 - val_accuracy: 0.9000 - lr: 1.0000e-04
Epoch 6/10
36/36 [==============================] - 8s 213ms/step - loss: 0.0101 - accuracy: 0.9964 - val_loss: 0.4656 - val_accuracy: 0.9250 - lr: 1.0000e-04
Epoch 7/10
36/36 [==============================] 

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.1956 - accuracy: 0.9500 - 286ms/epoch - 143ms/step
0.949999988079071


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.90097374 0.09902629]
 [0.9300481  0.06995189]
 [0.84474665 0.15525334]
 [0.85990894 0.14009102]
 [0.9102292  0.08977077]
 [0.95186305 0.04813693]
 [0.9285517  0.07144831]
 [0.5426651  0.45733482]
 [0.96571267 0.03428738]
 [0.9100207  0.08997931]
 [0.7001912  0.29980877]
 [0.92374754 0.07625248]
 [0.8988408  0.10115919]
 [0.96066153 0.03933845]
 [0.8231256  0.17687441]
 [0.8957328  0.10426713]
 [0.89892596 0.10107407]
 [0.81424975 0.1857502 ]
 [0.8861096  0.11389041]
 [0.9063584  0.09364158]
 [0.8753899  0.12461017]
 [0.8404787  0.1595213 ]
 [0.96739423 0.03260579]
 [0.9077466  0.09225336]
 [0.9077489  0.09225111]
 [0.8948809  0.10511913]
 [0.96909463 0.0309054 ]
 [0.89991    0.10009005]
 [0.918836   0.08116399]
 [0.87977463 0.12022536]
 [0.27564254 0.7243575 ]
 [0.21973976 0.7802602 ]
 [0.91308504 0.08691493]
 [0.8594385  0.1405615 ]
 [0.89010096 0.10989907]
 [0.88477933 0.11522061]
 [0.8810658  0.11893425]
 [0.8709853  0.12901469]
 [0.83834386 0.16165608]
 [0.7944391  0.20556092]]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#DenseNet Model

In [ ]:
densenet = DenseNet121(weights='imagenet',
                           include_top=False,
                           input_shape=(224,224,3)
                          )

29097984/29084464 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.models.Sequential([densenet,
                                        GlobalAveragePooling2D(),
                                        Dense(512, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.3),
                                        Dense(256, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.2),                                        
                                        Dense(128, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.1),
                                        Dense(2, activation='softmax')
                                      ])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 512)               524800    
                                                                 
 batch_normalization_5 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 256)              

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=15, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/15
36/36 [==============================] - 16s 233ms/step - loss: 0.2813 - accuracy: 0.8914 - val_loss: 0.0079 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
36/36 [==============================] - 4s 100ms/step - loss: 0.1479 - accuracy: 0.9457 - val_loss: 0.4006 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
36/36 [==============================] - 4s 100ms/step - loss: 0.1151 - accuracy: 0.9581 - val_loss: 0.0173 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/15
36/36 [==============================] - 4s 100ms/step - loss: 0.0761 - accuracy: 0.9768 - val_loss: 0.0253 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 5/15
36/36 [==============================] - 4s 101ms/step - loss: 0.0806 - accuracy: 0.9697 - val_loss: 0.0326 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6/15
36/36 [==============================] - 4s 102ms/step - loss: 0.0702 - accuracy: 0.9724 - val_loss: 0.0363 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 7/15
36/36 [==============================] - 4s 10

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.0312 - accuracy: 1.0000 - 180ms/epoch - 90ms/step
1.0


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[9.9806076e-01 1.9392088e-03]
 [6.8761164e-01 3.1238839e-01]
 [9.9671221e-01 3.2878001e-03]
 [9.5372754e-01 4.6272453e-02]
 [9.9282998e-01 7.1700118e-03]
 [9.9997807e-01 2.1911534e-05]
 [9.9999654e-01 3.4883437e-06]
 [6.0950875e-01 3.9049122e-01]
 [9.5429909e-01 4.5700911e-02]
 [9.9953532e-01 4.6465686e-04]
 [9.9941468e-01 5.8535917e-04]
 [9.5668608e-01 4.3313909e-02]
 [9.8759902e-01 1.2400919e-02]
 [9.9998844e-01 1.1530004e-05]
 [9.9926764e-01 7.3238160e-04]
 [9.9967766e-01 3.2239873e-04]
 [9.9826837e-01 1.7316311e-03]
 [9.7871792e-01 2.1282077e-02]
 [9.8354638e-01 1.6453600e-02]
 [9.9999797e-01 2.0400869e-06]
 [8.9794666e-01 1.0205335e-01]
 [1.0000000e+00 3.6379824e-08]
 [9.9970287e-01 2.9715791e-04]
 [9.9992430e-01 7.5730233e-05]
 [9.9997139e-01 2.8660288e-05]
 [9.9709082e-01 2.9091975e-03]
 [9.9820113e-01 1.7988287e-03]
 [9.9995720e-01 4.2820011e-05]
 [1.0000000e+00 1.2628619e-09]
 [9.7522926e-01 2.4770718e-02]
 [9.9162734e-01 8.3726309e-03]
 [9.9991560e-01 8.4392494e-05]
 [9.9998

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


# Inception V3

In [ ]:
inc_model = InceptionV3(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
print("number of layers in the model : ", len(inc_model.layers))

number of layers in the model :  311


In [ ]:
model = tf.keras.models.Sequential([inc_model,
                                        GlobalAveragePooling2D(),
                                    
                                        Dense(512, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.3),   
                                                                     
                                        Dense(2, activation='softmax')
                                      ])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_16 (Dense)            (None, 512)               1049088   
                                                                 
 batch_normalization_102 (Ba  (None, 512)              2048      
 tchNormalization)                                               
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 2)                

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=15, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/15
36/36 [==============================] - 11s 162ms/step - loss: 0.5265 - accuracy: 0.7943 - val_loss: 0.6379 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 2/15
36/36 [==============================] - 3s 76ms/step - loss: 0.3372 - accuracy: 0.8575 - val_loss: 0.0053 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
36/36 [==============================] - 3s 76ms/step - loss: 0.2947 - accuracy: 0.8753 - val_loss: 0.1735 - val_accuracy: 0.9250 - lr: 0.0010
Epoch 4/15
36/36 [==============================] - 3s 77ms/step - loss: 0.2567 - accuracy: 0.8896 - val_loss: 0.3753 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 5/15
36/36 [==============================] - 3s 76ms/step - loss: 0.2285 - accuracy: 0.9101 - val_loss: 0.5381 - val_accuracy: 0.7750 - lr: 1.0000e-04
Epoch 6/15
36/36 [==============================] - 3s 77ms/step - loss: 0.2433 - accuracy: 0.9101 - val_loss: 0.5288 - val_accuracy: 0.7750 - lr: 1.0000e-04
Epoch 7/15
36/36 [==============================] - 3s 77ms/st

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.6016 - accuracy: 0.7250 - 158ms/epoch - 79ms/step
0.7250000238418579


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.8029661  0.1970339 ]
 [0.48399207 0.51600796]
 [0.9480964  0.05190366]
 [0.9594473  0.04055272]
 [0.75508285 0.24491712]
 [0.4891471  0.5108529 ]
 [0.9831588  0.01684119]
 [0.16293065 0.83706933]
 [0.9650944  0.03490559]
 [0.9453138  0.05468616]
 [0.08657922 0.9134208 ]
 [0.9259774  0.07402261]
 [0.3723765  0.62762356]
 [0.82094944 0.17905061]
 [0.7925403  0.20745964]
 [0.83698237 0.1630176 ]
 [0.16899672 0.83100325]
 [0.64049286 0.35950714]
 [0.56138086 0.43861914]
 [0.69965565 0.30034438]
 [0.26889575 0.7311043 ]
 [0.80509514 0.19490488]
 [0.08081509 0.91918486]
 [0.20431639 0.79568356]
 [0.40050074 0.5994992 ]
 [0.5176611  0.48233894]
 [0.16922328 0.8307767 ]
 [0.85015255 0.1498475 ]
 [0.82432646 0.1756736 ]
 [0.98357856 0.0164214 ]
 [0.60986966 0.39013037]
 [0.9032411  0.09675892]
 [0.90635735 0.09364266]
 [0.6258972  0.3741027 ]
 [0.99014777 0.00985228]
 [0.53964925 0.46035078]
 [0.52825475 0.47174522]
 [0.5929079  0.4070921 ]
 [0.67748815 0.32251185]
 [0.87744725 0.1225528 ]]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#ResNet50 

In [ ]:
res = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model = tf.keras.models.Sequential([res,
                                        GlobalAveragePooling2D(),
                                    
                                        Dense(512, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.3),   
                                    
                                        Dense(256, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.2),  
                                    
                                        Dense(128, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.1),  
                                                                     
                                        Dense(2, activation='softmax')
                                      ])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_18 (Dense)            (None, 512)               1049088   
                                                                 
 batch_normalization_103 (Ba  (None, 512)              2048      
 tchNormalization)                                               
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 256)              

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=5, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/5
36/36 [==============================] - 10s 150ms/step - loss: 0.0474 - accuracy: 0.9760 - val_loss: 0.0658 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 2/5
36/36 [==============================] - 4s 101ms/step - loss: 0.0089 - accuracy: 0.9955 - val_loss: 0.0659 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 3/5
36/36 [==============================] - 4s 102ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.1301 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 4/5
36/36 [==============================] - 4s 104ms/step - loss: 0.0092 - accuracy: 0.9991 - val_loss: 0.1605 - val_accuracy: 0.9750 - lr: 1.0000e-04
Epoch 5/5
36/36 [==============================] - 4s 104ms/step - loss: 0.0054 - accuracy: 0.9982 - val_loss: 0.1780 - val_accuracy: 0.9750 - lr: 1.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.1780 - accuracy: 0.9750 - 179ms/epoch - 89ms/step
0.9750000238418579


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[9.99964476e-01 3.54786171e-05]
 [9.99699473e-01 3.00501153e-04]
 [9.99918342e-01 8.16123429e-05]
 [9.99996543e-01 3.42849580e-06]
 [9.99997735e-01 2.27045462e-06]
 [9.99999285e-01 7.01013334e-07]
 [1.00000000e+00 9.15822440e-09]
 [9.99857545e-01 1.42393823e-04]
 [9.99995947e-01 4.07468451e-06]
 [1.00000000e+00 3.17341015e-10]
 [9.99984145e-01 1.58245348e-05]
 [9.99999404e-01 5.62978244e-07]
 [9.99999881e-01 6.25440464e-08]
 [9.99999285e-01 7.50575737e-07]
 [9.99971867e-01 2.81457396e-05]
 [1.00000000e+00 4.33477698e-09]
 [9.99999762e-01 2.60708049e-07]
 [1.00000000e+00 4.97652197e-10]
 [9.99999762e-01 1.96905034e-07]
 [9.99898076e-01 1.01886006e-04]
 [9.99955058e-01 4.49548243e-05]
 [9.99999881e-01 1.66804170e-07]
 [9.99999881e-01 1.45932873e-07]
 [1.00000000e+00 4.34799308e-09]
 [9.99999762e-01 2.34979737e-07]
 [1.00000000e+00 5.91248899e-08]
 [9.99995351e-01 4.70684063e-06]
 [9.99999523e-01 5.14596991e-07]
 [1.00000000e+00 1.10020117e-08]
 [9.99967337e-01 3.26405461e-05]
 [8.082834

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#SSD Classifier

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_100 (Conv2D)         (None, 224, 224, 32)      896       
                                                                 
 conv2d_101 (Conv2D)         (None, 222, 222, 64)      18496     
                                                                 
 conv2d_102 (Conv2D)         (None, 220, 220, 128)     73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 110, 110, 128)    0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 110, 110, 128)     0         
                                                                 
 flatten_4 (Flatten)         (None, 1548800)           0         
                                                      

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=10, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/10
36/36 [==============================] - 16s 326ms/step - loss: 492.7135 - accuracy: 0.6955 - val_loss: 0.2959 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 2/10
36/36 [==============================] - 10s 288ms/step - loss: 0.7078 - accuracy: 0.7320 - val_loss: 0.4367 - val_accuracy: 0.4750 - lr: 0.0010
Epoch 3/10
36/36 [==============================] - 10s 286ms/step - loss: 0.4401 - accuracy: 0.8130 - val_loss: 0.0015 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/10
36/36 [==============================] - 10s 283ms/step - loss: 0.9218 - accuracy: 0.8175 - val_loss: 0.1316 - val_accuracy: 0.9750 - lr: 1.0000e-04
Epoch 5/10
36/36 [==============================] - 10s 283ms/step - loss: 0.3398 - accuracy: 0.8896 - val_loss: 0.1395 - val_accuracy: 0.9750 - lr: 1.0000e-04
Epoch 6/10
36/36 [==============================] - 10s 283ms/step - loss: 0.2680 - accuracy: 0.9012 - val_loss: 0.1061 - val_accuracy: 0.9750 - lr: 1.0000e-04
Epoch 7/10
36/36 [==============================] 

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.1082 - accuracy: 0.9500 - 167ms/epoch - 84ms/step
0.949999988079071


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[4.5649245e-01 5.4350752e-01]
 [1.0000000e+00 2.5582757e-14]
 [1.0000000e+00 1.6247316e-10]
 [9.9996328e-01 3.6725069e-05]
 [1.0000000e+00 5.7556049e-10]
 [1.0000000e+00 4.9200211e-27]
 [1.0000000e+00 1.4781114e-17]
 [8.7176166e-02 9.1282380e-01]
 [1.0000000e+00 6.3588312e-10]
 [1.0000000e+00 7.1784091e-21]
 [1.0000000e+00 6.1154230e-30]
 [1.0000000e+00 1.4758854e-10]
 [9.9996662e-01 3.3391792e-05]
 [1.0000000e+00 1.1598295e-31]
 [1.0000000e+00 2.3703741e-09]
 [1.0000000e+00 1.5138350e-15]
 [9.6511281e-01 3.4887202e-02]
 [1.0000000e+00 1.1244961e-25]
 [1.0000000e+00 1.4768589e-11]
 [1.0000000e+00 4.0808919e-21]
 [5.7070953e-01 4.2929053e-01]
 [1.0000000e+00 2.2001447e-33]
 [9.9997914e-01 2.0909547e-05]
 [1.0000000e+00 3.6458610e-11]
 [1.0000000e+00 0.0000000e+00]
 [9.9996054e-01 3.9488692e-05]
 [1.0000000e+00 5.6417343e-20]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 3.2387829e-15]
 [8.1934685e-01 1.8065317e-01]
 [7.8433448e-01 2.1566553e-01]
 [1.0000000e+00 2.5283684e-15]
 [9.9996

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#AlexNet

In [ ]:
model=Sequential()

#1 conv layer
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="valid",activation="relu",input_shape=(224,224,3)))

#1 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#2 conv layer
model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding="valid",activation="relu"))

#2 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#3 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#4 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#5 conv layer
model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#3 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())


model.add(Flatten())

#1 dense layer
model.add(Dense(4096,input_shape=(227,227,3),activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#2 dense layer
model.add(Dense(4096,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#3 dense layer
model.add(Dense(1000,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#output layer
model.add(Dense(2,activation="softmax"))

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=10, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/10
36/36 [==============================] - 4s 59ms/step - loss: 0.7554 - accuracy: 0.8130 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/10
36/36 [==============================] - 1s 31ms/step - loss: 0.3316 - accuracy: 0.8833 - val_loss: 7.1226 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 3/10
36/36 [==============================] - 1s 31ms/step - loss: 0.3267 - accuracy: 0.8825 - val_loss: 0.9416 - val_accuracy: 0.6750 - lr: 0.0010
Epoch 4/10
36/36 [==============================] - 1s 31ms/step - loss: 0.2057 - accuracy: 0.9261 - val_loss: 0.6070 - val_accuracy: 0.6750 - lr: 1.0000e-04
Epoch 5/10
36/36 [==============================] - 1s 30ms/step - loss: 0.1653 - accuracy: 0.9332 - val_loss: 0.7963 - val_accuracy: 0.6500 - lr: 1.0000e-04
Epoch 6/10
36/36 [==============================] - 1s 31ms/step - loss: 0.1517 - accuracy: 0.9386 - val_loss: 0.3293 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 7/10
36/36 [==============================] - 1s 31ms/

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.7455 - accuracy: 0.7250 - 36ms/epoch - 18ms/step
0.7250000238418579


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[9.5796847e-01 4.2031575e-02]
 [2.4541335e-02 9.7545868e-01]
 [9.9751377e-01 2.4862876e-03]
 [9.9799782e-01 2.0021363e-03]
 [9.9931622e-01 6.8384543e-04]
 [9.7678065e-01 2.3219356e-02]
 [9.9758077e-01 2.4192391e-03]
 [1.0026219e-01 8.9973778e-01]
 [4.7456499e-02 9.5254356e-01]
 [9.9777043e-01 2.2295562e-03]
 [4.4690812e-01 5.5309194e-01]
 [8.4616202e-01 1.5383802e-01]
 [9.9979502e-01 2.0501745e-04]
 [9.9996817e-01 3.1846375e-05]
 [8.5264742e-01 1.4735252e-01]
 [9.4757122e-01 5.2428804e-02]
 [9.8712212e-01 1.2877912e-02]
 [5.4075944e-01 4.5924053e-01]
 [3.5857296e-01 6.4142704e-01]
 [9.9914277e-01 8.5719221e-04]
 [6.5431967e-03 9.9345678e-01]
 [9.9999964e-01 3.1256451e-07]
 [9.5352393e-01 4.6476066e-02]
 [9.7529966e-01 2.4700299e-02]
 [4.3213564e-01 5.6786436e-01]
 [6.7902952e-01 3.2097048e-01]
 [9.2099667e-01 7.9003364e-02]
 [9.4325352e-01 5.6746561e-02]
 [9.9837106e-01 1.6289381e-03]
 [7.3355299e-01 2.6644707e-01]
 [3.2029310e-01 6.7970687e-01]
 [8.8118654e-01 1.1881342e-01]
 [9.9937

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#GoogleNet

In [ ]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

In [ ]:
def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (224, 224, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
  X1 = Dense(5, activation = 'softmax')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
  X2 = Dense(1000, activation = 'softmax')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
  X = Dense(2, activation = 'softmax')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

In [ ]:
model = GoogLeNet()

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=20, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/20
36/36 [==============================] - 13s 188ms/step - loss: 3.8421 - dense_32_loss: 0.9748 - dense_29_loss: 1.5567 - dense_31_loss: 1.3105 - dense_32_accuracy: 0.5646 - dense_29_accuracy: 0.6830 - dense_31_accuracy: 0.5619 - val_loss: 1.8955 - val_dense_32_loss: 0.5585 - val_dense_29_loss: 0.7535 - val_dense_31_loss: 0.5836 - val_dense_32_accuracy: 0.6750 - val_dense_29_accuracy: 0.5250 - val_dense_31_accuracy: 0.6750 - lr: 0.0010
Epoch 2/20
36/36 [==============================] - 4s 105ms/step - loss: 1.0530 - dense_32_loss: 0.3374 - dense_29_loss: 0.3377 - dense_31_loss: 0.3780 - dense_32_accuracy: 0.8736 - dense_29_accuracy: 0.8727 - dense_31_accuracy: 0.8673 - val_loss: 0.5894 - val_dense_32_loss: 0.1962 - val_dense_29_loss: 0.2105 - val_dense_31_loss: 0.1827 - val_dense_32_accuracy: 0.9500 - val_dense_29_accuracy: 0.9000 - val_dense_31_accuracy: 0.9250 - lr: 0.0010
Epoch 3/20
36/36 [==============================] - 4s 105ms/step - loss: 1.0000 - dense_32_loss: 0.3

In [ ]:
test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.6484 - dense_32_loss: 0.2354 - dense_29_loss: 0.1823 - dense_31_loss: 0.2306 - dense_32_accuracy: 0.9000 - dense_29_accuracy: 0.9000 - dense_31_accuracy: 0.9000 - 97ms/epoch - 49ms/step
[0.6483904123306274, 0.23543071746826172, 0.18233013153076172, 0.230629563331604, 0.8999999761581421, 0.8999999761581421, 0.8999999761581421]


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs[0],axis=1) #1-D array of classes

accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[array([[9.9608117e-01, 3.9188964e-03],
       [6.7337722e-02, 9.3266231e-01],
       [9.6283406e-01, 3.7165944e-02],
       [9.9908125e-01, 9.1876072e-04],
       [9.9808049e-01, 1.9194896e-03],
       [9.9920940e-01, 7.9057348e-04],
       [9.9973232e-01, 2.6761976e-04],
       [4.6284724e-02, 9.5371532e-01],
       [9.3729496e-01, 6.2704988e-02],
       [9.9945527e-01, 5.4476602e-04],
       [9.9022686e-01, 9.7730858e-03],
       [9.9723816e-01, 2.7618047e-03],
       [9.9617755e-01, 3.8224275e-03],
       [9.9986935e-01, 1.3062461e-04],
       [8.9825141e-01, 1.0174862e-01],
       [9.3602055e-01, 6.3979454e-02],
       [9.7638369e-01, 2.3616321e-02],
       [8.7209493e-01, 1.2790509e-01],
       [4.7495610e-01, 5.2504390e-01],
       [9.9958140e-01, 4.1858430e-04],
       [6.1748040e-01, 3.8251966e-01],
       [9.9934667e-01, 6.5332907e-04],
       [9.9920303e-01, 7.9694740e-04],
       [9.9932659e-01, 6.7345233e-04],
       [9.9938071e-01, 6.1930751e-04],
       [9.9816126e-01, 1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
